### This is work in progress

In [1]:
import torch
import torch.nn as nn
import egg.core as core

from torchvision import datasets, transforms
from torch import nn
from torch.nn import functional as F

import matplotlib.pyplot as plt
import random
import numpy as np

from pylab import rcParams
rcParams['figure.figsize'] = 5, 10

# For convenince and reproducibility, we set some EGG-level command line arguments here
opts = core.init(params=['--random_seed=7', # will initialize numpy, torch, and python RNGs
                         '--lr=1e-3',   # sets the learning rate for the selected optimizer 
                         '--batch_size=32',
                         '--optimizer=adam',])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import pickle
setting = 'context_unaware'
this_file = 'results/' + '(4,4)' + '_game_size_10_vsf_3/' + '/' + setting + '/' + str(2) + '/params.pkl'
with open(this_file, 'rb') as f:
    x = pickle.load(f)
    print(x)

Namespace(load_dataset=None, dimensions=[4, 4, 4, 4], attributes=3, values=4, game_size=10, vocab_size_factor=3, hidden_size=128, sender_cell='gru', receiver_cell='gru', learning_rate=0.001, temperature=2.0, length_cost=0.0, temp_update=0.99, save=True, num_of_runs=4, zero_shot=False, zero_shot_test=None, device=device(type='cuda'), path='', include_concept=False, context_unaware=True, random_seed=485109882, checkpoint_dir=None, preemptable=False, checkpoint_freq=0, validation_freq=1, n_epochs=300, load_from_checkpoint=None, no_cuda=False, batch_size=32, optimizer='adam', lr=0.01, update_freq=1, vocab_size=10, max_len=1, tensorboard=False, tensorboard_dir='runs/', distributed_port=18363, fp16=False, cuda=True, distributed_context=DistributedContext(is_distributed=False, rank=0, local_rank=0, world_size=1, mode='none'), game_setting='context_unaware', save_path='results\\(4,4)_game_size_10_vsf_3\\context_unaware\\2')


In [3]:
opts.game_size = 10
opts.hidden_size = 128
opts.vocab_size_factor = 3
opts.max_mess_len = None
opts.sender_cell = 'gru'
opts.receiver_cell = 'gru'
opts.temperature = 2
opts.length_cost = None

In [4]:
def loss(_sender_input, _message, _receiver_input, receiver_output, labels, _aux_input):
    """
    Loss needs to be defined for gumbel softmax relaxation.
    For a discriminative game, accuracy is computed by comparing the index with highest score in Receiver
    output (a distribution of unnormalized probabilities over target positions) and the corresponding 
    label read from input, indicating the ground-truth position of the target.
    Adaptation to concept game with multiple targets after Mu & Goodman (2021) with BCEWithLogitsLoss
        receiver_output: Tensor of shape [batch_size, n_objects]
        labels: Tensor of shape [batch_size, n_objects]
    """
    # after Mu & Goodman (2021):
    loss_fn = nn.BCEWithLogitsLoss()
    loss = loss_fn(receiver_output, labels)
    receiver_pred = (receiver_output > 0).float()
    per_game_acc = (receiver_pred == labels).float().mean(1).cpu().numpy()
    acc = per_game_acc.mean()
    return loss, {'acc': acc}

In [5]:
from egg.core.callbacks import CheckpointSaver
from archs import Sender, Receiver
import itertools

In [6]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [7]:
# setting
context_unaware = False # whether original or context_unaware simulations are evaluated
if context_unaware:
    setting = 'context_unaware'
    opts.context_unaware = True
else:
    setting = 'standard'
    opts.context_unaware = False

In [8]:
# initialize sender and receiver agents
opts.dimensions = list(itertools.repeat(n_values[0], n_attributes[0]))
sender = Sender(opts.hidden_size, sum(opts.dimensions), opts.game_size, opts.context_unaware)
receiver = Receiver(sum(opts.dimensions), opts.hidden_size)

minimum_vocab_size = opts.dimensions[0] + 1  # plus one for 'any'
vocab_size = minimum_vocab_size * opts.vocab_size_factor + 1  # multiply by factor plus add one for eos-symbol
print("vocab size", vocab_size)
# allow user to specify a maximum message length
if opts.max_mess_len:
    max_len = opts.max_mess_len
# default: number of attributes
else:
    max_len = len(opts.dimensions)
print("message length", max_len)

sender = core.RnnSenderGS(sender,
                            vocab_size,
                            int(opts.hidden_size / 2),
                            opts.hidden_size,
                            cell=opts.sender_cell,
                            max_len=max_len,
                            temperature=opts.temperature)

receiver = core.RnnReceiverGS(receiver,
                                vocab_size,
                                int(opts.hidden_size / 2),
                                opts.hidden_size,
                                cell=opts.receiver_cell)

game = core.SenderReceiverRnnGS(sender, receiver, loss, length_cost=opts.length_cost)

vocab size 16
message length 3


In [9]:
torch.load(paths[0]  + '/' + setting + '/' + str(0) + '/final.tar')

Checkpoint(epoch=300, model_state_dict=OrderedDict([('sender.sos_embedding', tensor([ 1.1216e-01, -1.3474e-01, -2.5066e-01, -1.8326e-02,  1.1909e-01,
        -1.7074e-01, -6.4368e-02,  2.6793e-02, -6.0732e-02, -2.3664e-01,
         5.7248e-02, -2.2886e-05, -9.9309e-03, -2.4520e-01,  2.6302e-02,
        -6.5818e-02, -1.5274e-01,  1.9002e-01, -2.6876e-01,  7.3717e-02,
        -7.8690e-03, -2.1576e-01, -1.7436e-02, -4.6681e-02,  1.3851e-01,
         6.6967e-02, -2.2197e-01, -1.2791e-01,  2.2565e-01,  1.8648e-01,
         2.4136e-01, -8.5576e-02, -3.1137e-03, -1.5894e-01,  8.2423e-02,
         2.4003e-01,  2.1567e-01, -9.3412e-02,  1.0525e-01, -1.5593e-01,
         6.1087e-02,  7.0872e-02, -3.5627e-02,  1.6416e-01, -2.4044e-01,
        -1.8552e-02,  1.2880e-01,  2.4741e-01,  1.0793e-01, -1.6862e-01,
         5.0780e-02, -1.7100e-01,  2.1746e-02, -4.4771e-02,  4.7373e-02,
         1.5273e-01,  1.4073e-02,  1.7372e-02, -1.5374e-01, -1.6908e-01,
        -1.7348e-02,  1.3752e-01, -7.8467e-02, 

In [17]:
game.state_dict = torch.load(paths[0]  + '/' + setting + '/' + str(0) + '/final.tar')

In [23]:
game.state_dict
# sender.sos_embedding
# sender.agent.fc1.weight
# sender.agent.fc1.bias
# sender.agent.fc2.weight
# sender.agent.fc2.bias
# ...
# sender.hidden_to_output.weight and bias
# sender.embedding.weight and bias
# sender.cell.weight_ih and hh and bias
# receiver.agent.fc1.weight
# receiver.agent.fc1.bias
# cell weights and biases
# embedding weights and biases
# optimizer_state_dict with states and 'exp_avg'
# param groups -> also sth. about optimizer

Checkpoint(epoch=300, model_state_dict=OrderedDict([('sender.sos_embedding', tensor([ 1.1216e-01, -1.3474e-01, -2.5066e-01, -1.8326e-02,  1.1909e-01,
        -1.7074e-01, -6.4368e-02,  2.6793e-02, -6.0732e-02, -2.3664e-01,
         5.7248e-02, -2.2886e-05, -9.9309e-03, -2.4520e-01,  2.6302e-02,
        -6.5818e-02, -1.5274e-01,  1.9002e-01, -2.6876e-01,  7.3717e-02,
        -7.8690e-03, -2.1576e-01, -1.7436e-02, -4.6681e-02,  1.3851e-01,
         6.6967e-02, -2.2197e-01, -1.2791e-01,  2.2565e-01,  1.8648e-01,
         2.4136e-01, -8.5576e-02, -3.1137e-03, -1.5894e-01,  8.2423e-02,
         2.4003e-01,  2.1567e-01, -9.3412e-02,  1.0525e-01, -1.5593e-01,
         6.1087e-02,  7.0872e-02, -3.5627e-02,  1.6416e-01, -2.4044e-01,
        -1.8552e-02,  1.2880e-01,  2.4741e-01,  1.0793e-01, -1.6862e-01,
         5.0780e-02, -1.7100e-01,  2.1746e-02, -4.4771e-02,  4.7373e-02,
         1.5273e-01,  1.4073e-02,  1.7372e-02, -1.5374e-01, -1.6908e-01,
        -1.7348e-02,  1.3752e-01, -7.8467e-02, 

In [11]:
game.eval()

r_input = torch.zeros(20, 12)
r_input[:10] = torch.Tensor(np.transpose(np.reshape(np.repeat([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1], 10), (12, 10))))
r_input[10:] = torch.Tensor(np.transpose(np.reshape(np.repeat([0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1], 10), (12, 10))))
m1 = torch.Tensor([2, 2, 2, 0])

with torch.no_grad():
    sample = game.receiver(m1).float().cpu()
    print(sample)
#for z in range(vocab_size):
#    t = torch.zeros(max_len + 1).to(device)
#    print(t)
#    with torch.no_grad():
#        # Receiver outputs a single tensor of predictions
#        sample = game.receiver(t).float().cpu()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x4 and 16x64)

In [12]:
torch.Tensor(np.transpose(np.reshape(np.repeat([0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1], 10), (12, 10))))

tensor([[0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1.]])

In [13]:
for d, data in enumerate(datasets):
    for run in range(5):
        #scores = pickle.load(open(paths[d] + '/' + setting + '/' + str(run) + '/entropy_scores.pkl', 'rb'))
        print(paths[d]  + '/' + setting + '/' + str(run) + '/final.tar')

results/(3,4)_game_size_10_vsf_3/standard/0/final.tar
results/(3,4)_game_size_10_vsf_3/standard/1/final.tar
results/(3,4)_game_size_10_vsf_3/standard/2/final.tar
results/(3,4)_game_size_10_vsf_3/standard/3/final.tar
results/(3,4)_game_size_10_vsf_3/standard/4/final.tar
results/(3,8)_game_size_10_vsf_3/standard/0/final.tar
results/(3,8)_game_size_10_vsf_3/standard/1/final.tar
results/(3,8)_game_size_10_vsf_3/standard/2/final.tar
results/(3,8)_game_size_10_vsf_3/standard/3/final.tar
results/(3,8)_game_size_10_vsf_3/standard/4/final.tar
results/(3,16)_game_size_10_vsf_3/standard/0/final.tar
results/(3,16)_game_size_10_vsf_3/standard/1/final.tar
results/(3,16)_game_size_10_vsf_3/standard/2/final.tar
results/(3,16)_game_size_10_vsf_3/standard/3/final.tar
results/(3,16)_game_size_10_vsf_3/standard/4/final.tar
results/(4,4)_game_size_10_vsf_3/standard/0/final.tar
results/(4,4)_game_size_10_vsf_3/standard/1/final.tar
results/(4,4)_game_size_10_vsf_3/standard/2/final.tar
results/(4,4)_game_size